# Домашнее задание к лекции "Базовые понятия статистики"

Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [2]:
# Список имен столбцов
col_names = [
    'surgery?', 'Age', 'Hospital Number', 'rectal temperature',
    'pulse', 'respiratory rate', 'temperature of extremities',
    'peripheral pulse', 'mucous membranes', 'capillary refill time',
    'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube',
    'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination',
    'abdomen', 'packed cell volume', 'total protein',
    'abdominocentesis appearance', 'abdomcentesis total protein',
    'outcome', 'surgical lesion?', 'type of lesion 01',
    'type of lesion 02', 'type of lesion 03', 'cp_data',
]
len(col_names)

28

In [3]:
# Загружаем в датафрейм данные из восьми столбцов по условию:
horse_data = pd.read_csv(
    'stat_datas/horse_data.csv',
    header=0,
    names = col_names
)[['surgery?', 'Age', 'rectal temperature',
  'pulse', 'respiratory rate', 'temperature of extremities',
  'pain', 'outcome']]
horse_data.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,1,39.2,88,20,?,3,3
1,2,1,38.30,40,24,1,3,1
2,1,9,39.10,164,84,4,2,2
3,2,1,37.30,104,35,?,?,2
4,2,1,?,?,?,2,2,1


In [4]:
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   surgery?                    299 non-null    object
 1   Age                         299 non-null    int64 
 2   rectal temperature          299 non-null    object
 3   pulse                       299 non-null    object
 4   respiratory rate            299 non-null    object
 5   temperature of extremities  299 non-null    object
 6   pain                        299 non-null    object
 7   outcome                     299 non-null    object
dtypes: int64(1), object(7)
memory usage: 18.8+ KB


## Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

### Проблема №1
В датафрейме пропущенные значения представлены в виде '?'

In [5]:
# заменим все '?' в датафрейме на NaN:
horse_data = horse_data.replace(['?'], np.NaN)
horse_data.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,1,39.2,88,20,NaN,3,3
1,2,1,38.30,40,24,1,3,1
2,1,9,39.10,164,84,4,2,2
3,2,1,37.30,104,35,NaN,NaN,2
4,2,1,NaN,NaN,NaN,2,2,1


### Проблема №2
В датафрейме имеется ошибочно определены типы данных в столбцах 'Age', 'rectal temperature', 'pulse', 'respiratory rate'

In [6]:
# преобзаруем типы данных значений выше перечисленных столбцов:
horse_data['Age'] = horse_data['Age'].astype(object)
horse_data['rectal temperature'] = horse_data['rectal temperature'].astype(float)
horse_data[['pulse', 'respiratory rate']] = horse_data[['pulse', 'respiratory rate']].astype('Int64')

In [7]:
# Проверяем:
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    298 non-null    object 
 1   Age                         299 non-null    object 
 2   rectal temperature          239 non-null    float64
 3   pulse                       275 non-null    Int64  
 4   respiratory rate            241 non-null    Int64  
 5   temperature of extremities  243 non-null    object 
 6   pain                        244 non-null    object 
 7   outcome                     298 non-null    object 
dtypes: Int64(2), float64(1), object(5)
memory usage: 19.4+ KB


### Анализ значений по столбцам

#### surgery?

In [8]:
# определим, какие значения и в камом количестве присутствуют в столбце
horse_data['surgery?'].value_counts()

1    180
2    118
Name: surgery?, dtype: int64

In [9]:
# рассчитаем моду:
horse_data['surgery?'].mode()[0]

'1'

**Вывод:** Хирургическое лечение использовалось чаще (в 180 случаях), а консервативное в 118.

#### Age

In [10]:
# определим, какие значения и в камом количестве присутствуют в столбце
horse_data['Age'].value_counts()

1    275
9     24
Name: Age, dtype: int64

В столбце 'Age' имеются значения '1' и '9'. Это не соответствует документации к базе данных.

In [11]:
# Заменим значения '9' на '2':
horse_data['Age'] = horse_data['Age'].replace(9, 2)

In [12]:
horse_data['Age'].value_counts()

1    275
2     24
Name: Age, dtype: int64

In [13]:
# Рассчитаем моду:
horse_data['Age'].mode()[0]

1

**Вывод:** В исследование включены 275 взрослых животных и 24 молодых (в возрасте до 6 месяцев)

#### rectal temperature

In [14]:
# рассмотрим основные статистики
horse_data['rectal temperature'].describe()

count    239.000000
mean      38.166527
std        0.733508
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [15]:
# определим выбросы, используя межквартильный размах (iqr):
iqr = horse_data['rectal temperature'].quantile(0.75) - horse_data['rectal temperature'].quantile(0.25)
lower_bound = horse_data['rectal temperature'].quantile(0.25) - (iqr * 1.5)
upper_bound = horse_data['rectal temperature'].quantile(0.75) + (iqr * 1.5)
outliers = [x for x in horse_data['rectal temperature'] if x <= lower_bound or x >= upper_bound]
print(*outliers)

39.9 35.4 40.3 39.7 36.4 40.3 39.6 36.5 36.0 36.1 36.6 40.8 40.0 36.5


**Вывод:** Распределение значений в столбце rectal temperature близко к нормальному. Рассчитанные значения выбросов могут вполне соответствовать значениям, полученным при исследовании животных. Поэтому удаление этих значений не целесообразно.

#### pulse

In [16]:
# посчитаем основные статистики:
horse_data['pulse'].describe()

count        275.0
mean     71.934545
std      28.680522
min           30.0
25%           48.0
50%           64.0
75%           88.0
max          184.0
Name: pulse, dtype: Float64

In [17]:
# определим выбросы, используя межквартильный размах (iqr):
iqr = horse_data['pulse'].quantile(0.75) - horse_data['pulse'].quantile(0.25)
lower_bound = horse_data['pulse'].quantile(0.25) - (iqr * 1.5)
upper_bound = horse_data['pulse'].quantile(0.75) + (iqr * 1.5)

In [18]:
# выведем строки со значениями по столбцу 'pulse', определенными как выбросы
horse_data[~ horse_data['pulse'].between(lower_bound, upper_bound, inclusive='both')]

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
2,1,2,39.1,164,84,4,2,2
40,2,2,39.0,150,72,NaN,NaN,1
54,1,2,38.6,160,20,3,3,2
254,1,2,38.8,184,84,1,4,2
274,1,2,38.8,150,50,1,5,2


**Вывод:** Значения столбца 'pulse', которые были определены как выбросы, в полной мере могут соответствовать реальным значениям, полученным при проведении исследования. Это подтверждают высокие знаяения ректарльной температуры в этих же строках. Поэтому удалять строки с этими значениями не целесообтазно.

#### respiratory rate

In [19]:
horse_data['respiratory rate'].describe()

count        241.0
mean     30.427386
std      17.678256
min            8.0
25%           18.0
50%           24.0
75%           36.0
max           96.0
Name: respiratory rate, dtype: Float64

In [20]:
# определим выбросы, используя межквартильный размах (iqr):
iqr = horse_data['respiratory rate'].quantile(0.75) - horse_data['respiratory rate'].quantile(0.25)
lower_bound = horse_data['respiratory rate'].quantile(0.25) - (iqr * 1.5)
upper_bound = horse_data['respiratory rate'].quantile(0.75) + (iqr * 1.5)

In [21]:
# выведем строки со значениями по столбцу 'respiratory rate', определенными как выбросы
horse_data[~ horse_data['respiratory rate'].between(lower_bound, upper_bound, inclusive='both')]

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
2,1,2,39.1,164,84,4,2,2
38,1,2,39.2,146,96,NaN,NaN,2
40,2,2,39.0,150,72,NaN,NaN,1
81,1,2,38.1,100,80,3,3,1
83,1,1,37.8,60,80,1,2,1
102,1,2,38.0,140,68,1,3,1
105,1,1,38.3,52,96,NaN,NaN,1
119,1,1,39.4,54,66,1,2,1
124,1,1,38.0,42,68,4,3,1
185,1,1,39.3,64,90,2,NaN,1


**Вывод:** В ряде случаев значения частоты дыхания значительно превышают норму (8 - 10). Однако такие случаи часто сочетаются с выраженными изменениями по показателям rectal temperature и puls. Поэтому надо внимательнее отнестить к этим "выбросам" и сто раз подумать, прежде чем удалять эти случаи из выборки.

#### temperature of extremities

In [22]:
# определим, какие значения и в камом количестве присутствуют в столбце
horse_data['temperature of extremities'].value_counts()

3    108
1     78
2     30
4     27
Name: temperature of extremities, dtype: int64

In [23]:
horse_data['temperature of extremities'].mode()[0]

'3'

**Вывод:** В большинстве случаев (108) температура конечностей расценивалась как "прохладные".

#### pain

In [24]:
# определим, какие значения и в камом количестве присутствуют в столбце
horse_data['pain'].value_counts()

3    67
2    59
5    41
4    39
1    38
Name: pain, dtype: int64

In [25]:
horse_data['pain'].mode()[0]

'3'

**Вывод:** В большинстве случаес боль оценивалась как "перемежающаяся легкая боль"

#### outcome

In [26]:
# определим, какие значения и в камом количестве присутствуют в столбце
horse_data['outcome'].value_counts()

1    178
2     76
3     44
Name: outcome, dtype: int64

In [27]:
horse_data['outcome'].mode()[0]

'1'

**Вывод:** Чаще (178 случвев) наступало выздоровление животных.

## Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

#### Рассчет количества пропусков

In [28]:
# рассчитаем % пропусков по столбцам
(horse_data.isna().mean() * 100).round(2)

surgery?                       0.33
Age                            0.00
rectal temperature            20.07
pulse                          8.03
respiratory rate              19.40
temperature of extremities    18.73
pain                          18.39
outcome                        0.33
dtype: float64

In [29]:
horse_data.isna().sum()

surgery?                       1
Age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

#### Удаление пропусков из столбцов surgery? и outcome

In [30]:
horse_data[horse_data['surgery?'].isna()]

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
131,NaN,1,38.0,48,20,3,4,NaN


По одному пропуску имеется в столбцах 'surgery?' и 'outcome'. Причем оба этих пропуска находятся в одно строке.
**Вывод:** Можно удалить эту строку без особых последствий.

In [31]:
# удаляем строку с пропусками в столбце 'surgery?'
horse_data = horse_data.dropna(subset='surgery?')

#### Замена пропусков в столбце rectal temperature

In [32]:
horse_data.groupby('outcome')['rectal temperature'].median()

outcome
1    38.20
2    38.05
3    38.05
Name: rectal temperature, dtype: float64

**Вывод:** В связи с тем, что во всех группах исхода лечения знаяения медианы ректальной температуры практически одинаковы, все пропуски в столбце моно заполнить значением медианы

In [33]:
# замена пропусков медианой
horse_data['rectal temperature'] = horse_data['rectal temperature'].fillna(horse_data['rectal temperature'].median())

#### Замена пропусков в столбце pulse

In [34]:
horse_data.groupby('outcome')['pulse'].median()

outcome
1    54.0
2    86.0
3    82.0
Name: pulse, dtype: Float64

In [35]:
horse_data.groupby(['outcome', 'Age'])['pulse'].median()

outcome  Age
1        1       52.0
         2      120.0
2        1       84.0
         2      141.0
3        1       81.0
         2      100.0
Name: pulse, dtype: Float64

Учитывая выраженную разницу значений менианы пульса в различных группах по исходу и возрасту, заполним пропуски в столбце pulse значениями медиан, соответствующими группировке.

In [36]:
# замена пропусков медианой в соответствии с группировкой
horse_data['pulse'] = horse_data['pulse'].fillna(horse_data.groupby(['outcome', 'Age'])['pulse'].transform('median'))

#### Замена пропусков в столбце respiratory rate

In [37]:
horse_data.groupby('outcome')['respiratory rate'].median()

outcome
1    24.0
2    30.0
3    24.0
Name: respiratory rate, dtype: Float64

In [38]:
horse_data.groupby('Age')['respiratory rate'].median()

Age
1    24.0
2    49.0
Name: respiratory rate, dtype: Float64

In [39]:
horse_data.groupby(['outcome', 'Age'])['respiratory rate'].median()

outcome  Age
1        1      24.0
         2      64.0
2        1      28.0
         2      44.0
3        1      24.0
         2      <NA>
Name: respiratory rate, dtype: Float64

**Вывод:** Наиболее выражена разница в значениях медиан частоты дыхания между возрастными группами. Поэтому произведем замену медианами в зависимости от возрастной группы.

In [40]:
# замена пропусков медианой в соответствии с группировкой по возрасту
horse_data['respiratory rate'] = horse_data['respiratory rate'].fillna(horse_data.groupby(['Age'])['respiratory rate'].transform('median'))

#### Замена пропусков в столбце temperature of extremities

In [41]:
horse_data.groupby(['outcome', 'temperature of extremities'])['temperature of extremities'].count()

outcome  temperature of extremities
1        1                             63
         2                             26
         3                             46
         4                             11
2        1                             11
         2                              3
         3                             40
         4                              9
3        1                              4
         2                              1
         3                             21
         4                              7
Name: temperature of extremities, dtype: int64

In [42]:
horse_data.groupby(['Age', 'temperature of extremities'])['temperature of extremities'].count()

Age  temperature of extremities
1    1                              71
     2                              29
     3                             100
     4                              26
2    1                               7
     2                               1
     3                               7
     4                               1
Name: temperature of extremities, dtype: int64

**Вывод:** Произведем замену пропущенных значений в столбце temperature of extremities на значения мод в зависимости от группы по признаку outcome

In [60]:
# заменим пустые значения в группе выживших животных
horse_data.loc[horse_data.outcome == '1', 'temperature of extremities'] = horse_data.loc[horse_data.outcome == '1', 'temperature of extremities'].fillna(horse_data.loc[horse_data.outcome == '1', 'temperature of extremities'].mode()[0])

In [61]:
# заменим пустые значения в группах погибших и усыпленных животных
horse_data.loc[horse_data.outcome.isin(['2', '3']), 'temperature of extremities'] = horse_data.loc[horse_data.outcome.isin(['2', '3']), 'temperature of extremities'].fillna(horse_data.loc[horse_data.outcome.isin(['2', '3']),  'temperature of extremities'].mode()[0])

#### Замена пропусков в столбце pain

In [64]:
horse_data.pain.value_counts()

3    67
2    59
5    41
4    38
1    38
Name: pain, dtype: int64

In [63]:
horse_data.groupby(['outcome', 'pain'])['pain'].count()

outcome  pain
1        1       35
         2       33
         3       54
         4       10
         5       12
2        1        2
         2       14
         3        8
         4       17
         5       23
3        1        1
         2       12
         3        5
         4       11
         5        6
Name: pain, dtype: int64

In [65]:
horse_data.groupby(['Age', 'pain'])['pain'].count()

Age  pain
1    1       37
     2       54
     3       62
     4       35
     5       38
2    1        1
     2        5
     3        5
     4        3
     5        3
Name: pain, dtype: int64

**Вывод:** В столюце pain логичнее заменить пропущенные значения наиболее часто встречающимся значением

In [68]:
# заменим пустые значения столбца модой
horse_data.pain = horse_data.pain.fillna(horse_data.pain.mode()[0])

#### Итоговый датафрейм не содержит пропущенных значений

In [70]:
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    298 non-null    object 
 1   Age                         298 non-null    int64  
 2   rectal temperature          298 non-null    float64
 3   pulse                       298 non-null    Int64  
 4   respiratory rate            298 non-null    Int64  
 5   temperature of extremities  298 non-null    object 
 6   pain                        298 non-null    object 
 7   outcome                     298 non-null    object 
dtypes: Int64(2), float64(1), int64(1), object(4)
memory usage: 21.5+ KB
